# Módulo 3
**Desarrollo de proyectos de análisis de datos  IN1002B**

### Pronósticos: Series de tiempo

**Base de datos:**

This dataset offers a valuable resource for businesses operating in the retail furniture sector. By analyzing historical sales data from the superstore dataset, users can gain insights into future sales patterns and trends. This information can be utilized to optimize inventory management strategies, anticipate customer demand, and enhance overall operational efficiency. Whether for retail managers, analysts, or data scientists, this dataset provides a foundation for informed decision-making, helping businesses maintain stability and drive sustained growth in the dynamic retail environment.

[Kaggle](https://www.kaggle.com/datasets/tanayatipre/store-sales-forecasting-dataset)



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
# read data
data = pd.read_csv('', encoding='latin-1')

In [ ]:
#head()


In [ ]:
#info()


In [7]:
# Formato de fecha

# Columna 1
data['Order Date'] = pd.to_datetime(data['Order Date'], format='mixed')

# Columna 2


In [ ]:
#info()


In [9]:
## Años y meses

#Años
data['Year'] = data['Order Date'].dt.year

#Meses


In [ ]:
# unique para ver que años tenemos


Como lo hemos visto en las sesiones de visualizacion anteriores, los gráficos de línea son ideales para observar cambios a traves del tiempo.

EStos cambios pueden deberse a diversos factores, uno de los más conocidos es la estacionalidad.

En el siguiente ejemplo vamos a graficar con la función ```sns.lineplot()```, agregaremos en ```hue= ''``` los años que comprende el estudio y por último cambiaremos las etiquetas de eje.

In [ ]:
# vamos a graficar cada año
plt.figure(figsize=(10, 6))
sns.lineplot(data=data, x='Month', y='Profit', hue='Year', marker='o', errorbar=None)

# Configuración de etiquetas
plt.xticks(ticks=range(1, 13), labels=['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio',
                                          'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre'])
plt.title('Profit por Mes y Año')
plt.xlabel('Mes')
plt.ylabel('Profit')
plt.legend(title='Año')
plt.grid()

Como pudieron ver, diversas razones no nos permiten visualizar las diferencias entre cada año, algunos pueden ser:

- La similitud de patrones
- La paleta de colores
- etc.


Es por eso que utilizaremos la función ```FacetGrid()```, la cuál nos permite realizar un mosaico de gráficos, lo único que requerimos es:
1. Base de datos
2. Número de columnas
3. Ramaño de las figuras


La diferencia también radica en que trabajaremos por capas, para ello es necesario guardar nuestra imagen en una variable:

In [ ]:
# Crear un gráfico facetado
g = sns.FacetGrid(data, col='Year', col_wrap=2, height=4, aspect=1.5)
g.map(sns.lineplot, 'Month', 'Profit', marker='o', errorbar=None)

# Configuración de etiquetas
g.set(xticks=range(1, 13))
g.set_xticklabels(['Ene', 'Feb', 'Mar', 'Abr', 'May', 'Jun',
                   'Jul', 'Ago', 'Sept', 'Oct', 'Nov', 'Dic'])
g.set_titles(col_template='{col_name}')
g.set_axis_labels('Mes', 'Profit')
g.add_legend()

# Mostrar el gráfico
plt.show()

## **Moving Average** (MA)

Una media móvil es un cálculo utilizado para analizar un conjunto de datos en modo de puntos para crear series de promedios. Así las medias móviles son una lista de números en la cual cada uno es el promedio de un subconjunto de los datos originales

El primer ejemplo muestra el cálculo de media movil de los 4 años que abarca el dataset.

### Pronóstico: 2018

In [ ]:
mean_profit = data.groupby('Year')['Profit'].mean().reset_index()
print(mean_profit)

In [14]:
# Renombramos las columnas
mean_profit.columns = ['Year', 'Average Profit']

In [ ]:
# Pronóstico para 2018 usando el promedio de los años anteriores
average_forecast_2018 = mean_profit['Average Profit'].mean()
print(average_forecast_2018)

In [ ]:
# Crear un nuevo DataFrame para el año x
new_data = pd.DataFrame({'Year': ['2018'], 'Average Profit': [average_forecast_2018]})
print(new_data)

In [ ]:
# Concatenar el DataFrame original con el nuevo año
final_df = pd.concat([mean_profit, new_data], ignore_index=True)

# Mostrar el resultado final
print(final_df)

**Segunda opción**

Existe una función llamada ```rolling``` que funciona a traves de ventanas. Es decir, si la ventana es de 4 y tus datos son [1, 2, 3, 4, 5], las ventanas serán:

    [1, 2, 3, 4]
    [2, 3, 4, 5]

Posteriormente, puedes hacer calculos, tales como media, mediana, moda, mínimo y máximo. Lo cual nos permite simplificar el primer código y replicar la lógica:

In [ ]:
mean_profit = data.groupby('Year')['Profit'].mean().reset_index()

mean_profit = mean_profit['Profit'].rolling(window=4).mean()

mean_profit

### Pronóstico: Enero 2018

Para este ejemplo vamos a considerar solo los datos obtenidos durante los meses del año 2017:

In [19]:
dic2017 = data[data['Year'] == 2017]

Y repetimos los pasos anteriores para obtener el pronostico para enero del 2018:

In [ ]:
mean_profit = dic2017.groupby('Month')['Profit'].mean().reset_index()

mean_profit.columns = ['Month', 'Average Profit']

# Pronóstico para Enero del 2019 utilizando meses anteriores
average_forecast_ene = mean_profit['Average Profit'].mean()

# Crear un nuevo DataFrame para incluir enero del 2018
new_data = pd.DataFrame({'Month': ['Ene-18'], 'Average Profit': [average_forecast_ene]})

# Concatenar el DataFrame original con el nuevo año
final_df = pd.concat([mean_profit, new_data], ignore_index=True)

# Mostrar el resultado final
print(final_df)

También hacemos el ejemplo con el método de ```rolling``` para validar los resultados:

In [ ]:
mean_profit = dic2017.groupby('Month')['Profit'].mean().reset_index()

mean_profit = mean_profit['Profit'].rolling(window=12).mean()

mean_profit

## **Promedio Movil Ponderado**

En el promedio móvil ponderado podemos asignar cualquier importancia (peso) a cualquier dato del promedio (siempre que la sumatoria de las ponderaciones sean equivalentes al 100%). Es una práctica regular aplicar el factor de ponderación (porcentaje) mayor al dato más reciente. [Referencia](https://ingenieriaindustrialonline.com/pronostico-de-la-demanda/promedio-movil-ponderado/)



In [ ]:
mean_profit = data.groupby('Year')['Profit'].mean().reset_index()
mean_profit.columns = ['Year', 'Average Profit']

# Definir los pesos para los años (más reciente tiene mayor peso)
pesos = [0.1, 0.2, 0.3, 0.4]  # Pesos para 2014, 2015, 2016 y 2017

# Calcular el promedio móvil ponderado para los años relevantes
if len(mean_profit) >= 3:
    weighted_average = (mean_profit['Average Profit'].iloc[-1] * pesos[0] +
                        mean_profit['Average Profit'].iloc[-2] * pesos[1] +
                        mean_profit['Average Profit'].iloc[-2] * pesos[2] +
                        mean_profit['Average Profit'].iloc[-3] * pesos[3])
else:
    weighted_average = mean_profit['Average Profit'].mean()  # Si hay menos de 3 años

# Crear un nuevo DataFrame para el año 2018
new_data = pd.DataFrame({'Year': [2018], 'Average Profit': [weighted_average]})

# Concatenar el DataFrame original con el nuevo año
final_df = pd.concat([mean_profit, new_data], ignore_index=True)

# Mostrar el resultado final
print(final_df)

Revisar

In [ ]:
mean_profit = dic2017.groupby('Month')['Profit'].mean().reset_index()
mean_profit.columns = ['Month', 'Average Profit']

# Definir los pesos para los últimos meses
pesos = [0.05, 0.05, 0.05, 0.05, 0.1, 0.1, 0.1, 0.1,  0.1, 0.1, 0.3, 0.4]  # Pesos para los últimos 4 meses

# Filtrar los últimos meses
if len(mean_profit) >= len(pesos):
    last_months = mean_profit['Average Profit'].iloc[-len(pesos):]
    weighted_average = sum(last_months.iloc[i] * pesos[i] for i in range(len(pesos)))
else:
    weighted_average = mean_profit['Average Profit'].mean()  # Si hay menos meses

# Crear un nuevo DataFrame para el pronóstico del próximo año (2018)
new_data = pd.DataFrame({'Month': ['Next Year'], 'Average Profit': [weighted_average]})

# Concatenar el DataFrame original con el nuevo año
final_df = pd.concat([mean_profit, new_data], ignore_index=True)

# Mostrar el resultado final
print(final_df)

### **¿Cuándo utilizar un pronóstico de promedio móvil ponderado?**

El pronóstico de promedio móvil ponderado es óptimo para patrones de demanda aleatorios o nivelados donde se pretende eliminar el impacto de los elementos irregulares históricos mediante un enfoque en períodos de demanda reciente, dicho enfoque es superior al del promedio móvil simple.

In [ ]:
import numpy as np

def generar_vector(n):
    # Generar n-1 elementos aleatorios pequeños entre 0 y 0.1
    vector = np.random.rand(n - 1) * 0.1

    # Agregar el último elemento como un valor mayor
    vector = np.append(vector, 0.9)

    # Normalizar para que la suma sea 1
    vector /= vector.sum()

    return vector

# Tamaño del vector
n = 12
resultado = generar_vector(n)
print(resultado)
print("Suma:", resultado.sum())
